#### If the math looks weird run code below and restart the notebook.

In [7]:
%pip install mathjax


# Traveling Salesman Problem (TSP)

## Introduction

> The travelling salesman problem (TSP) seeks to the find the minimum cost hamiltonian circuit in an undirected graph, $G_n$.

> Or more generally, given a list of $n$ cities, finds the fastest way to visit every city once and then return to the first city.

### Problem Formulation

> Let $I$ be a set of $n$ cities

> Then, define an arc $C_{i,j}$ where $i,j\in I$ exists if it is possible to travel from city $i$ to city $j$. And is weighted by the travel time from city $i$ to city $j$.

> Next, let $X_{i,j}$ be a variable such that,
$$ X_{i,j} = \begin{cases}
                1&\text{City $i$ connects to City $j$ is in the hamiltonian circuit}
                \\0&\text{Otherwise}
            \end{cases} $$

> In a traditional TSP, it is possible to travel from any city to any other city, making a $n$ complete graph, $K_n$. $\\$
This means there are $n \choose 2$ total arcs. $\\$
And so there are, 
$$
    |H(K_n)| = \frac{1}{2}(n-1)!
$$ 

>total hamiltonian cycles in $K_n$ $\\$

> This means that a brute force solution runs in exponential time, and is generally NP-Complete.

> This means that the brute force solution has poor scalability, in fact for higher values of $n$ the solution could take years. $\\$
Critically; however, the problem can be cast as a network optimization problem from an adaptation of minimum spanning trees, which is potentially capable of solving the problem much faster.


### Integer Programming Model Local Constraints

> Here we cover the basic formulation for a minimum tour problem.

#### Parameters

> Let $I$ for a list of $n$ cities.

> Let $C_{i,j}$ be the cost of moving from city $i$ to city $j$ (in travel time by road route).

#### Variables :

> Let $X_{i,j}\in\{0,1\}$ be a binary variable determining if arc $C_{i,j}$ is used in the Hamiltonian circuit.
$$ X_{i,j} = \begin{cases}
    1&\text{City $i$ connects to City $j$ is in the hamiltonian circuit}
    \\0&\text{Otherwise}
\end{cases} $$
> Then the goal is to find the least cost way to visit every city starting and returning to city $1$.

#### Objective Function

>Let the cost of any circuit be,
$$
    \min\sum_{i\in I}\sum_{j\in I}C_{i,j}X_{i,j}
$$

#### Local Constraints

> Subject to the constraints,

>> Entering every city only once: 
$$
    \sum_{i \in I}X_{i,j}=1, \quad\quad \forall j\in I
$$

>> Leaving every city only once:
$$
    \sum_{k\in I}X_{j,k}=1, \quad\quad \forall j\in I
$$

>> Don't stay at one city:
$$
    X_{i,i} = 0, \quad\quad \forall i\in I
$$

### Sub Tour Problems

> Unfortunately, the local constraints above are not restrictive enough to eliminate all solutions that do not generate Hamiltonian circuits. In particular, it is still possible to get sub-tours that contribute to the final solution

![Image_of_sub-tours_Didn't_load_correctly...](subTour_Image.png)



> Therefore, extra constraints should be added to prevent sub-tours in the final solution.

## Miller-Tucker-Zemlin 1960 (MTZ) Sub-tour Elimination

> One of the best methods to eliminate sub-tours is using the Miller-Tucker-Zemlin (MTZ) approach. Which adds a new time variable $t$, indicating at which point each city is visited in sequence.

#### New Parameter

> First fix some ordering on the list of cities $I$, and let $V = \{1,2,\cdots,n\}$ be that order. So for some $i\in V$, $i = 1,2,\cdots,n$ for $n$ cities. $\\$
Then, for $i\in V$ let $i$ denote either an integer, or a city $i\in I$.

#### Variable

> Next define the time variable, $t_i$ by the time in sequence that city $i$ is visited.

> As a result,
$$
    \text{if } X_{i,j}=1, \quad \text{ then } \quad t_j \geq t_i+1, \quad\quad i,j\neq 1
$$

> This abuses notation a bit, but in general the $i,j$ associated with $X_{i,j}$ are city names. And, the $i,j$ associated with $t_j,t_i$ are integers associated with the ordering on $I$,

> Notice intuitively, this works by restricting a sequence on the order we visit each city. Or rather if we visit cities in the order, $i_1,i_2,\cdots,i_{n-1}$ then,
$$
t_{i_1}=1,t_{i_2}=2,t_{i_3}=3,\cdots,t_{i_{n-1}}=n-1,
$$

> Then, suppose a sub tour exists, such that, $\exists a,b,c\in I$ where $X_{a,b}=X_{b,c}=X_{c,a}=1$ (cycle $C_3$). and none of the cities $a,b,c$ are the originating city. ($a,b,c\neq 1$). $\\$
Then, the time sequence constraints impose,
$$
    \begin{cases}
        t_b \geq t_a+1\\
        t_c \geq t_b+1\\
        t_a \geq t_c+1
    \end{cases}
$$

> Which has no solution. Similar logic can be applied to any sub-tour, ($C_k$), and so this method effectively eliminates sub-tours from the optimal solution.

> Lastly, consider how to encode the 'if' statement, 
$$
    \text{if } X_{i,j}=1, \quad \text{ then } \quad t_j \geq t_i+1, \quad\quad i,j\neq 1
$$

>As a constraint

> In particular, consider,

$$
\begin{cases}
    t_j \geq t_i+1 &X_{i,j}=1\\
    t_j \text{ Unrestricted }& X_{i,j} = 0
\end{cases}
$$

> And so consider the following constraint,
$$
    t_j \geq t_i+1-M(1-X_{i,j})
$$

> For large $M$.

> Actually because it has been shown that there exists an ordering that ensures 
$$
    \max_{i\in V}t_i = n-1
$$

> Then selecting $M=n$ is sufficient to guarantee that an optimal Hamiltonian Circuit can satisfy the constraints.

## Final Problem Formulation

### Parameters

> $I$ be a set of $n$ cities.

> $V$ be an ordering on $I$ such that, $i\in I$ is either an integer $i$, or the $i'th$ city in the ordering of $I$.

> Let $C_{i,j}$ be the cost of moving from city $i$ to city $j$ (in travel time by road route).

### Variables

> Let $X_{i,j}\in \{0,1\}$ be a binary variable if we travel from city $i$ to city $j$ in the tour.
$$ X_{i,j} = \begin{cases}
    1&\text{City $i$ connects to City $j$ is in the hamiltonian circuit}
    \\0&\text{Otherwise}
\end{cases} $$

> Let $t_i$ be the time city $i$ is visited in the tour. For $i\in V$ is an integer.

### Objective Function

>Let the cost of any circuit be,
$$
    \min\sum_{i\in I}\sum_{j\in I}C_{i,j}X_{i,j}
$$

### Local Constraints

>> Entering every city only once: 
$$
    \sum_{i \in I}X_{i,j}=1, \quad\quad \forall j\in I
$$

>> Leaving every city only once:
$$
    \sum_{k\in I}X_{j,k}=1, \quad\quad \forall j\in I
$$

>> Don't stay at one city:
$$
    X_{i,i} = 0, \quad\quad \forall i\in I
$$

### MTZ Constraints

>> Sub-tour elimination
$$
    t_j \geq t_i+1-n(1-X_{i,j}), \quad\quad \forall i,j\in V, \quad j>i
$$

>> Circuit completeness
$$
    t_1 = 1
$$




> The following is a file to run a TSP problem with MTZ constraint solution method. Using default parameters all results are already included in the project package. And so running any portion of the the following is optional.

# Instructions

This file contains automation for running the Traveling Salesman Problem (TSP) program for user selected cities.

Note: this file uses a lot of python dependencies and API's and so is dependent on API keys and package installs.
Importantly, all cells generate a file that is already included in the project folder, and so unless you want to change
the default cities, running every cell is not necessary.

On a high level each section does the following:

    >section: 'city information' >>collects which cities to use for the TSP (you can update this list)

    >section: 'Data Collection' >>converts the list of city information to longitude latitude coordinates,
                                then calculates the travel time between all city pairs. Making a $K_n$ complete
                                graph. (for $n$ cities).
                                Importantly, the length of each arc, $C_{i,j}$ is the travel time from city $i$
                                to city $j$, based on the best estimated road distance between cities.
                                And so cities that do not have feasible driving routes will not be able to 
                                be connected.
                                Data is collected from DistanceMatrix.ai API, https://distancematrix.ai/

    >section: 'AMPL solution' >>first generates a new .dat file using the distance matrix from the previous section.
                                Then, solves the TSP using Miller-Tucker-Zemlin (MTZ) method.
                                The result of which is printed to file 'TSP_MTZ_TOUR_Result.txt'.

    >section: 'Display result' >>uses the file generated from the AMPL solution to print the optimal tour, and display
                                the tour on the US map.

# Dependencies

> You will probably need to restart the kernal after install dependencies.

In [8]:
#  Pip install dependencies....
%pip install pip
%pip install mathjax
%pip install docopt==0.6.2
%pip install geographiclib==2.0
%pip install geopy==2.3.0
%pip install matplotlib==3.6.2
%pip install numpy==1.23.3
%pip install pandas==1.5.1
%pip install plotly==5.11.0
%pip install python-dateutil==2.8.2
%pip install python-dotenv==0.21.0
%pip install requests==2.28.1
%pip install urllib3==1.26.12
%pip install amplpy
%pip install os

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


# City Information

> This section selects which cities to use as inputs. Listed are the default cities, and so all pre-set files corespond to this list.
You can change the cities in this list and all default files will be adjusted as the notebook runs.

In [1]:
from main import *

#  Add more cities if you want (The API is slow so it might take a while to collect data)
cities = [
    'Denver',
    'New York',
    'Houston',
    'Dallas',
    'Philadelphia',
    'Phoenix',
    'Miami',
    'Cleveland',
    'San Francisco',
    'Nashville',
    'Greensboro',
    'Lincoln',
    'Seattle'
    ]
print(cities)

ModuleNotFoundError: No module named 'src'

# Data Generation

> This section prepares information from cities list.

> First by creating a list of longitude/latitude coordinates for each city.

> Then creating a distance matrix for each city pair.
(city A , city B) -> C_{A,B} = travel time from city A to city B

> Travel times are based on the optimal driving route from city A to city B. 
This is collected from DistanceMatrix.ai API. https://distancematrix.ai/.

> This information is then stored in a JSON file called 'DistanceMatrix.json'
Which is read by AMPL wrapper to solve the TSP problem.


>> Importantly, sometimes the API may not be able to find a route between distant cities, in this case, that corresponding arc is neglected by getting an untractable value.
$$
    C_{i,j}^{neglected} = 100000 \text{ minutes }
$$

> This removes the arc from any optimal tour. If it looks like this affects the optimal solution, run the API request again and it will probably find a route for the neglected arc.

In [10]:
# Convert City strings to coordinate location (Takes a while)
locations = {}
for city in cities :
    locations[city] = LocationToCoordinate(city)
print(locations)

{'Denver': (39.7392364, -104.984862), 'New York': (40.7127281, -74.0060152), 'Houston': (29.7589382, -95.3676974), 'Dallas': (32.7762719, -96.7968559), 'Philadelphia': (39.9527237, -75.1635262), 'Phoenix': (33.4484367, -112.074141), 'Miami': (25.7741728, -80.19362), 'Cleveland': (41.4996574, -81.6936772), 'San Francisco': (37.7790262, -122.419906), 'Nashville': (36.1622767, -86.7742984), 'Greensboro': (36.0726355, -79.7919754), 'Lincoln': (40.8088861, -96.7077751), 'Seattle': (47.6038321, -122.330062)}


In [11]:
# Plots cities
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import nbformat
df = {}
for name, coord in locations.items() :
    df[name] = {'lat' : coord[0], 'long' : coord[1], 'color' : 0.75, 'size' : 0.5, 'name' : name}

fig = px.scatter_geo(df.values(),lat='lat',lon='long', color = 'color', size = 'size', range_color = [0,1], hover_name = 'name')

fig.update_geos(fitbounds="locations",scope="usa")
fig.update_layout(title = 'US map', title_x=0.5)
fig.show()

### The following cell calculates the drive time duration between every possible city pair $(i,j)$

> The API is very slow, and takes about $10$ minutes for $10$ cities.

> If you kept the default cities, you do not need to run this again, because the results JSON, 'DistanceMatrix.json' is already included in the project folder.

> Also, this API is generated from my personal API key, which corresponds to a free account. The API key is included in a protected .env file and automatically retrieved. I don't know how well this will work after distributing the repository. So if it doesn't work.... only the default cities can be used.

In [ ]:
# Calculate distance between all cities  (Takes about 10 minutes for 10 cities)
# distance metric is based on the road route between two cities
# specifically is the time in minutes it takes to drive between any
# two cities. This is based on DistanceMatrix.ai open API. (Which hopefully works...)
DistanceMatrix = citiesToDistanceMatrix(locations)
for city, info in DistanceMatrix.items() :
    print(city + ': ' + str([(name,travel_info['duration']) for name, travel_info in info.items()]) + '\n')


>> 

> Saves the distance matrix to a file, 'DistanceMatrix.json'. To be read by the AMPL solver in the next section.

In [ ]:
# Writes a distance matrix to a json filename.
# Later read by AMPL wrapper.
if DistanceMatrix != None :
    writeToJSON(DistanceMatrix, filename = 'DistanceMatrix.json')
#print(JsonToDict('DistanceMatrix.json'))  # Verifies that the file was created correctly and is readable.

### Display Complete graph

Displays all derived arcs between the all cities.

In [14]:
# Plots cities
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import nbformat

tour_info_lat = []
tour_info_long = []
for i,city_A in enumerate(cities) :
    for city_B in cities[i:] :
        tour_info_lat += [locations[city_A][0]]
        tour_info_lat += [locations[city_B][0]]
        tour_info_lat += [None]
        tour_info_long += [locations[city_A][1]]
        tour_info_long += [locations[city_B][1]]
        tour_info_long += [None]

fig = go.Figure(data=go.Scattergeo(
    lat = tour_info_lat,
    lon = tour_info_long,
    mode = 'lines',
    line = dict(width = 2, color = 'blue'),
))

fig.update_geos(fitbounds="locations",scope="usa")
fig.update_layout(title = 'TSP all arcs on US map', title_x=0.5)
fig.show()

NOTE: lines between cities are shown as linear (straight segments) however, are weighted by road travel routing information. I don't have a good way to get the geometry of a route just how long it takes to travel.

# AMPL Solution

> This section runs an AMPL terminal from the notebook. (It may or may not work... if it doesn't, the AMPL files will still be generated and can be run from AMPLIDE).

> Frist, AMPL_Wrapper section will make a .dat file called 'TSP_DATA.dat' from the distance matrix JSON file.
This will be used by the AMPL solution solver.

> The next section starts the AMPL terminal and runs (TSP_MTZ.mod , TSP_DATA.dat , TSP_MTZ.run).

> Importantly, in this section, the variable 

>> path_to_ampl_EXE_Folder

> References the path to the folder holding AMPL.exe. Because this is just a wrapper it needs access to AMPL from local computer.

> This means to run the AMPL files from the notebook you will need to link your own path to the folder holding AMPL.exe.

>> path_to_ampl_EXE_Folder = r'Path to folder holding AMPL.exe'

> Also, the full output from the AMPL terminal will likely not load fully in the notebook.
However, it will provide an option to open the output in a new text editor. 

In [15]:
# Makes an AMPL .dat file from the JSON distance matrix data
from AMPL_Wrapper import *

makeDatFile(data = 'DistanceMatrix.json', outfile = 'TSP_DATA.dat')

#### Connect wrapper to AMPL.exe

Here to use AMPL on the notebook you need to link to your AMPL application folder.

 And so in the variable 'path_to_ampl_EXE_Folder' place your local path to the folder containing AMPL.exe.

>> >> path_to_ampl_EXE_Folder = r'Path to folder holding AMPL.exe'

Then run the cell.

> Or you can just run the AMPL files directly from AMPL IDE, outside of the notebook. (They will create a file called TSP_MTZ_TOUR_Results.txt that can be read by the rest of the notebook.)

In [16]:
from amplpy import AMPL
from amplpy import Environment

#  The following runs ampl commands to find shortest paths. You will need to include your
#  file path to the folder containing ampl.exe, or you can just run the .dat/.mod/.run files
#  in the ampl ide. (This notebook just tries to automate the processes in one entity)

path_to_ampl_EXE_Folder = r'C:\Users\IanSi\Downloads\amplide.mswin64\ampl.mswin64'

ampl = AMPL(Environment(path_to_ampl_EXE_Folder))

# Runs ampl files.
#  Note it is likely that the terminal result will not be fully
#  displayable in the notebook, and so it will make a new textfile
#  that contains the full result.
#  This file, TSP_MTZ.run also creates a txt file output with the 
#  result (the smallest tour.) This text file is called 'TSP_MTZ_TOUR_Result.txt'
ampl.read('TSP_MTZ.mod')
ampl.read_data('TSP_DATA.dat')
ampl.read('TSP_MTZ.run')


CPLEX 20.1.0.0: mipdisplay 2
MIP Presolve modified 24 coefficients.
Reduced MIP has 170 rows, 168 columns, and 732 nonzeros.
Reduced MIP has 156 binaries, 0 generals, 0 SOSs, and 0 indicators.
Found incumbent of value 18158.800000 after 0.01 sec. (0.71 ticks)
Probing time = 0.00 sec. (0.34 ticks)
Cover probing fixed 0 vars, tightened 12 bounds.
Detecting symmetries...
MIP Presolve modified 24 coefficients.
Reduced MIP has 170 rows, 168 columns, and 732 nonzeros.
Reduced MIP has 156 binaries, 0 generals, 0 SOSs, and 0 indicators.
Probing time = 0.00 sec. (0.35 ticks)
Cover probing fixed 0 vars, tightened 12 bounds.
Clique table members: 92.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.02 sec. (0.38 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0     

# Display Results

>This section displays the result from the AMPL solver.

>It will both print a list of the final tour,

> and display an image of the tour on a US map

In [17]:
#  Prints the tour result from the AMPL solution

city_info = []
with open('TSP_MTZ_TOUR_Result.txt', 'r') as f:
    for line in f :
        city_info += [tuple(line.split())]
city_info.sort(key = lambda x: int(x[1]))

ordered_tour_names = []
print('TOUR')
tour_string = ''
for name,i in city_info :
    tour_string += name + ' --> '
    ordered_tour_names += [name]
ordered_tour_names += [city_info[0][0]]
tour_string += city_info[0][0]
print(tour_string)


TOUR
Denver --> Lincoln --> Nashville --> Cleveland --> New_York --> Philadelphia --> Greensboro --> Miami --> Houston --> Dallas --> Phoenix --> San_Francisco --> Seattle --> Denver


In [18]:
# Displays Optimal Tour

# Plots cities
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import nbformat
df = {}
for name, coord in locations.items() :
    df[name] = {'lat' : coord[0], 'long' : coord[1], 'color' : 0.75, 'size' : 0.5, 'name' : name}
tour_info = {}
for city in ordered_tour_names :
    tour_info[city.replace("_", " ")] = df[city.replace("_", " ")]

tour_info['final'] = df[ordered_tour_names[0]]


fig = px.line_geo(tour_info.values(),lat='lat',lon='long', color = 'color', hover_name = 'name', markers=True)

fig.update_geos(fitbounds="locations",scope="usa")
fig.update_layout(title = 'TSP MTZ solution on US map', title_x=0.5)
fig.show()

NOTE: lines between cities are shown as linear (straight segments) however, are weighted by road travel routing information. I don't have a good way to get the geometry of a route just how long it takes to travel.

# Traditional TSP (Only using Local Constraints)

The following includes AMPL solution information for running TSP using only local constraints so solutions with sub-tours can be compared to MTZ solutions.

In [19]:
from amplpy import AMPL
from amplpy import Environment

#  The following runs ampl commands to find shortest paths. You will need to include your
#  file path to the folder containing ampl.exe, or you can just run the .dat/.mod/.run files
#  in the ampl ide. (This notebook just tries to automate the processes in one entity)

ampl = AMPL(Environment(path_to_ampl_EXE_Folder))

# Runs ampl files.
#  Note it is likely that the terminal result will not be fully
#  displayable in the notebook, and so it will make a new textfile
#  that contains the full result.
#  This file, TSP_traditional.run also creates a txt file output with the 
#  result (the smallest tour.) This text file is called 'TSP_traditional_TOUR_Result.txt'
ampl.read('TSP_traditional.mod')
ampl.read_data('TSP_DATA.dat')
ampl.read('TSP_traditional.run')

CPLEX 20.1.0.0: mipdisplay 2
Reduced MIP has 26 rows, 156 columns, and 312 nonzeros.
Reduced MIP has 156 binaries, 0 generals, 0 SOSs, and 0 indicators.
Found incumbent of value 18374.766667 after 0.02 sec. (0.48 ticks)
Probing time = 0.00 sec. (0.20 ticks)
Reduced MIP has 26 rows, 156 columns, and 312 nonzeros.
Reduced MIP has 156 binaries, 0 generals, 0 SOSs, and 0 indicators.
Probing time = 0.00 sec. (0.20 ticks)
Clique table members: 26.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.09 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                        18374.7667        0.0000           100.00%
*     0+    0                        17166.0500        0.0000           100.00%
*     0+    0                        15262.2333        0.000

Collects list of tuples for city pairs

In [20]:
#  Prints the tour result from the AMPL solution

city_info = []
with open('TSP_traditional_TOUR_Result.txt', 'r') as f:
    for line in f :
        city_info += [tuple(line.split())]

tour_info = {}
for city_A, city_B in city_info :
    coord_A = LocationToCoordinate(city_A.replace("_", " "))
    coord_B = LocationToCoordinate(city_B.replace("_", " "))
    tour_info[ city_A.replace("_", " ") + ' ORIG' ] = {'lat' : coord_A[0], 'long' : coord_A[1], 'name' : city_A.replace("_", " "), 'color' : 0.75, 'size' : 0.5}
    tour_info[ city_A.replace("_", " ") + ' DEST' ] = {'lat' : coord_B[0], 'long' : coord_B[1], 'name' : city_B.replace("_", " "), 'color' : 0.75, 'size' : 0.5}
    tour_info[ city_A.replace("_", " ") + ' none' ] = {'lat' : None, 'long' : None, 'name' : None, 'color' : 0.75, 'size' : 0.5}

print('TOUR')
tour_string = ''
for city_pair in city_info :
    tour_string += str(city_pair) + '\t'
print(tour_string)


TOUR
('Denver', 'Lincoln')	('New_York', 'Philadelphia')	('Houston', 'Dallas')	('Dallas', 'Houston')	('Philadelphia', 'New_York')	('Phoenix', 'Seattle')	('Miami', 'Greensboro')	('Cleveland', 'Nashville')	('San_Francisco', 'Phoenix')	('Nashville', 'Cleveland')	('Greensboro', 'Miami')	('Lincoln', 'Denver')	('Seattle', 'San_Francisco')	


Displays Tour solution on US Map.

In [21]:
# Displays Optimal Tour

# Plots cities
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import nbformat

fig = go.Figure(data=go.Scattergeo(
    lat = [info['lat'] for info in tour_info.values()],
    lon = [info['long'] for info in tour_info.values()],
    mode = 'lines',
    line = dict(width = 2, color = 'blue')
))

fig.update_geos(fitbounds="locations",scope="usa")
fig.update_layout(title = 'TSP local constraints solution on US map', title_x=0.5)
fig.show()

NOTE: lines between cities are shown as linear (straight segments) however, are weighted by road travel routing information. I don't have a good way to get the geometry of a route just how long it takes to travel.

The above display shows the result of the local constraint TSP. Notice, the result is likely to have sub-tours.